<a href="https://colab.research.google.com/github/mavropulokn/functionsHw/blob/main/Python_Advanced_%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_(HW).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 15.0 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 


Опишем поведение агента, всегда играющего "камень" - это значение 0

In [25]:
%%writefile ablaze.py
import numpy as np

ablaze_combination_action = 0


def ablaze(observation, configuration):
    """Gets action based on random selection (acts every configuration.signs steps, ablaze cycle).

    After configuration.signs steps (combination) a cycle repeats.
    Uses the ablaze_combination_action saved during each cycle.
    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action, new action selected during ablaze cycle
    """

    # Action to repeat in series (combination) of steps.
    global ablaze_combination_action

    if observation.step % configuration.signs == 0:
        ablaze_combination_action = np.random.randint(0, configuration.signs)
    else:
        return ablaze_combination_action


Overwriting ablaze.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [26]:
%%writefile anti_counter_attack_last_move.py
from collections import deque

import numpy as np

anti_counter_attack_last_move_agent_auxiliary_marker = 1
anti_counter_attack_last_move_agent_moves = deque(maxlen=2)
opponent_moves = deque(maxlen=1)


def anti_counter_attack_last_move(observation, configuration):
    """Gets action to win the counter_attack_last_move agent.

    Detects counter_attack_last_move agent (tested in 3 consecutive rounds).
    Uses the queue to save last 2 actions and mark to save the criteria check result.
    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action to win counter_attack_last_move agent or random one (in detection criteria isn't verified)
    """

    # Queue to save last 2 actions.
    global anti_counter_attack_last_move_agent_moves

    # Mark to save the criteria validation.
    global anti_counter_attack_last_move_agent_auxiliary_marker

    def is_counter_attack_last_move_agent_move_expected_equals_actual(configuration, observation,
                                                                      counter_attack_defencemoves):
        """Gets predicate or to check if the opponent could be counter_attack_last_move agent."""
        return observation.lastOpponentAction == (
                counter_attack_defencemoves.popleft() + 1) % configuration.signs

    def act_against_counter_attack_last_move_agent(anti_counter_attack_last_move_agent_moves, configuration):
        """Gets action against counter_attack_last_move agent, saves action and returns."""

        action = (anti_counter_attack_last_move_agent_moves.popleft() + 2) % configuration.signs
        anti_counter_attack_last_move_agent_moves.append(action)
        return action

    def is_counter_attack_last_move_agent_detected(anti_counter_attack_last_move_agent_auxiliary_marker,
                                                   observation):
        """Checks the counter_attack_last_move_agent detection criteria has succeeded.

        Criteria is the 3 equal actual and expected actions, should be after 5th step.
        So we could act against the detected counter_attack_last_move agent.
        """
        return anti_counter_attack_last_move_agent_auxiliary_marker == 1 and observation.step >= 5

    def is_counter_attack_last_move_agent_detection_in_progress(anti_counter_attack_last_move_agent_auxiliary_marker,
                                                                observation):
        return anti_counter_attack_last_move_agent_auxiliary_marker == 1 and observation.step >= 2

    if is_counter_attack_last_move_agent_detected(anti_counter_attack_last_move_agent_auxiliary_marker,
                                                  observation):
        return act_against_counter_attack_last_move_agent(anti_counter_attack_last_move_agent_moves,
                                                          configuration)

    elif is_counter_attack_last_move_agent_detection_in_progress(anti_counter_attack_last_move_agent_auxiliary_marker,
                                                                 observation):
        if is_counter_attack_last_move_agent_move_expected_equals_actual(configuration, observation,
                                                                         anti_counter_attack_last_move_agent_moves):
            move = np.random.randint(0, configuration.signs)
            anti_counter_attack_last_move_agent_moves.append(move)
        else:
            anti_counter_attack_last_move_agent_auxiliary_marker = 0
    elif anti_counter_attack_last_move_agent_auxiliary_marker == 1:
        # Can't compare the calculated and actual opponents move before second step,so just save.
        move = np.random.randint(0, configuration.signs)
        anti_counter_attack_last_move_agent_moves.append(move)
    else:
        # Marker equals 0, so just return random, opponent probably is not the counter_attack_last_move agent.
        move = np.random.randint(0, configuration.signs)

        return move


Overwriting anti_counter_attack_last_move.py


In [48]:
%%writefile anti_mode.py
import numpy as np

anti_mode_auxiliary_marker = 1
anti_mode_dictionary = dict().fromkeys([0, 1, 2], 0)


def anti_mode(observation, configuration):
    """Gets action against mode agent.

    Uses the anti_mode_dictionary to save the frequency distribution.
    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action to win mode agent action or random one (in detection criteria isn't verified)
    """

    """True if the anti_mode agent has been detected by this agent."""
    global anti_mode_auxiliary_marker

    """Dictionary to keep the agent moves."""
    global anti_mode_dictionary

    def get_mode_agent_action_expected(configuration, mode_defence_dictionary):
        """Gets opponent expected action."""
        opponent_action_expected = (max(mode_defence_dictionary,
                                        key=mode_defence_dictionary.get) + 1) % configuration.signs
        return opponent_action_expected

    def is_mode_agent_detection_in_progress(mode_defence_auxiliary_marker, observation):
        """Gets predicate to check if the criteria is not failed on the current step."""
        return observation.step > 1 and mode_defence_auxiliary_marker

    def is_mode_agent_detected(mode_defence_auxiliary_marker, observation):
        """Gets predicate to check if mode attack agent criteria already verified."""
        return observation.step > 4 and mode_defence_auxiliary_marker == 1

    def get_action_against_mode_agent(anti_mode_dictionary, configuration):
        """Gets action against mode agent."""
        return (max(anti_mode_dictionary, key=anti_mode_dictionary.get) + 2) % configuration.signs

    if is_mode_agent_detected(anti_mode_auxiliary_marker, observation):
        move = get_action_against_mode_agent(anti_mode_dictionary, configuration)
        anti_mode_dictionary[move] += 1
        return move
    elif is_mode_agent_detection_in_progress(anti_mode_auxiliary_marker, observation):
        opponents_expected_action = get_mode_agent_action_expected(configuration, anti_mode_dictionary)
        if observation.lastOpponentAction == opponents_expected_action:
            move = np.random.randint(0, configuration.signs)
            anti_mode_dictionary[move] += 1
            return move
        else:
            anti_mode_auxiliary_marker = 0
            return np.random.randint(0, configuration.signs)
    else:
        move = np.random.randint(0, configuration.signs)
        anti_mode_dictionary[move] += 1
        return move


Writing anti_mode.py


In [27]:
%%writefile anti_uniform_move.py
import numpy as np

uniform_move_detector_auxiliary_marker = 1
opponent_move = None


def anti_uniform_move(observation, configuration):
    """Gets action to win the similarly (uniformly) acting agent.

    Detects uniform agent in 3 consecutive rounds.
    Detects if the opponent uniform behaviour changes after detection criteria succeeded and does random moves.
    Uses the mark to save the criteria check result.
    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action to win uniformly acting agent or random one (in detection criteria isn't verified)
    """

    """True if the uniformly acting agent (like rock or scissors agent) has been detected by this agent."""
    global uniform_move_detector_auxiliary_marker

    """Keeps opponent move."""
    global opponent_move

    def is_uniformly_acting_agent_detected(observation, opponent_move,
                                           uniform_move_detector_auxiliary_marker):
        """Gets predicate to check if uniform_move_detector agent criteria verified (3 similar moves in a row)."""
        return observation.step > 4 and uniform_move_detector_auxiliary_marker \
               and observation.lastOpponentAction == opponent_move

    def opponent_is_not_uniformly_acting(observation, opponent_move, uniform_move_detector_auxiliary_marker):
        """Gets predicate to check if opponent_is_not_uniformly_acting after first step."""
        return observation.step > 1 and uniform_move_detector_auxiliary_marker \
               and observation.lastOpponentAction != opponent_move

    if is_uniformly_acting_agent_detected(observation, opponent_move,
                                          uniform_move_detector_auxiliary_marker):
        # action against agent with the same actions on every step
        return (opponent_move + 1) % configuration.signs
    elif opponent_is_not_uniformly_acting(observation, opponent_move, uniform_move_detector_auxiliary_marker):
        uniform_move_detector_auxiliary_marker = 0
    elif observation.step == 1:
        opponent_move = observation.lastOpponentAction

    return np.random.randint(0, configuration.signs)


Overwriting anti_uniform_move.py


In [52]:
%%writefile arithmetic_average_attack.py
import numpy as np

arithmetic_average_attack_sum = 0
arithmetic_average_attack_count = 0


def arithmetic_average_attack(observation, configuration):
    """Gets action based on arithmetic last moves average estimate.

    Uses the counters and cumulative sum saved on each step.
    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action, arithmetic average, or random one on the very first move
    """

    """Keeps arithmetic average attack sum."""
    global arithmetic_average_attack_sum

    """Keeps arithmetic average attack sum."""
    global arithmetic_average_attack_count

    if observation.step > 0:
        arithmetic_average_attack_sum += observation.lastOpponentAction
        arithmetic_average_attack_count += 1
        return int(arithmetic_average_attack_sum / arithmetic_average_attack_count)
    else:
        return np.random.randint(0, configuration.signs)


Overwriting arithmetic_average_attack.py


In [29]:
%%writefile counter_attack_last_move.py
import numpy as np


def counter_attack_last_move(observation, configuration):
    """Gets action to win the last opponents move.

    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action to win last opponent move or random if the first round
    """

    def get_action_against(configuration, observation):
        """ Gets action against last opponent action."""
        return (observation.lastOpponentAction + 1) % configuration.signs

    if observation.step > 0:
        return get_action_against(configuration, observation)
    else:
        return np.random.randint(0, configuration.signs)


Overwriting counter_attack_last_move.py


In [30]:
%%writefile mode.py
import numpy as np

mode_dictionary = dict().fromkeys([0, 1, 2], 0)


def mode(observation, configuration):
    """Gets action based on the most frequent opponent move.

    Uses the mode_dictionary to save the frequency distribution.
    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action to win opponents moves distribution mode or random one (if not enough data)
    """

    """Keeps opponent moves frequency distribution."""
    global mode_dictionary

    def get_action_against_opponent_moves_distribution_mode(mode_dictionary, configuration):
        """Gets action against opponent moves by the mode estimate."""
        mode_value = max(mode_dictionary, key=mode_dictionary.get)
        return (mode_value + 1) % configuration.signs

    if observation.step > 1:
        opponent_action = observation.lastOpponentAction
        mode_dictionary[opponent_action] += 1
        return get_action_against_opponent_moves_distribution_mode(mode_dictionary, configuration)
    else:
        return np.random.randint(0, configuration.signs)


Overwriting mode.py


In [31]:
%%writefile opponent_imitator.py
import numpy as np


def opponent_imitator(observation, configuration):
    """Gets action from opponent last move (imitator).

    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action, opponent last move or random if the first round
    """

    if observation.step == 0:
        return np.random.randint(0, configuration.signs)
    else:
        return observation.lastOpponentAction


Overwriting opponent_imitator.py


In [32]:
%%writefile paper.py
def paper(observation, configuration):
    """Gets action 1 (so-called paper).

    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: 1
    """

    return 1


Overwriting paper.py


In [33]:
%%writefile rock.py
def rock(observation, configuration):
    """Gets action 0 (so-called rock).

    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: 0
    """

    return 0


Overwriting rock.py


In [34]:
%%writefile scissors.py
def scissors(observation, configuration):
    """Gets action 2 (so-called scissors).

    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: 2
    """

    return 2


Overwriting scissors.py


In [35]:
%%writefile straight.py
def straight(observation, configuration):
    """Gets action based on the step.

    Returns the remainder of division of the step on configuration.signs.
    So we have ascending combination, similar to straight hand in poker.
    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int, action calculated from the step
    """

    return observation.step % configuration.signs


Overwriting straight.py


In [36]:
%%writefile uniform_random.py
import numpy as np


def uniform_random(observation, configuration):
    """Gets uniformly distributed value.

    This rock-paper-scissors comparison functionality is so-called agent.
    Agent could be used for competition by the kaggle_environments framework.
    :param observation: observation object with auxiliary data
    :param configuration: configuration object with game configuration
    :return: int action, random move
    """

    return np.random.randint(0, configuration.signs)


Overwriting uniform_random.py


In [58]:
class DuplicateAgentException(Exception):
    pass


class Tournament:
    """This is the Tournament class implemented as the directed weighted graph."""

    def __init__(self):
        """Constructor.

        :param self: self
        """

        self.map = dict()

    def get_tournament_table(self):
        """Gets tournament table state (directed weighted graph).

        :param self: self
        :return: dictionary, tournament directed weighted graph
        """

        return self.map

    def add_agent(self, agent):
        """Adds agent to tournament.

        :param self: self
        :param agent: agent to add (identifier)
        """

        if agent in self.map:
            raise DuplicateAgentException("Duplicate agent")
        self.map[agent] = dict()

    def get_agent(self, agent):
        """Gets agent from tournament table.

        :param self: self
        :param agent: agent identifier or name
        :return: dictionary of agent games
        """

        return self.map[agent]

    def add_game_auxiliary(self, one, another, score):
        """Adds game result to the tournament for one side of the game.

        :param self: self
        :param one: one agent
        :param another: another agent
        :param score: score, positive, is the one agent is winner and negative if looser
        """

        if one not in self.map:
            self.add_agent(one)
        agent_map = self.get_agent(one)
        agent_map[another] = Game(one, another, score)

    def add_game(self, one, another, score):
        """Adds game result to the tournament.

        :param self: self
        :param one: one agent
        :param another: another agent
        :param score: score, positive, is the one agent is winner and negative if looser
        """

        self.add_game_auxiliary(one, another, score)
        self.add_game_auxiliary(another, one, -score)

    def get_winners_details(self):
        """Gets array of sorted by agent score descending tuples.

        First value in each tuple item is agent name.
        The second is the map of agent games with the data to get the sum.
        :return: array of sorted tuples, tournament result with details
        """

        return sorted(self.map.items(), key=lambda item: sum([z.score for z in item[1].values()]), reverse=True)

    def get_winners(self):
        """Gets array of sorted by agent score descending tuples.

        First value in each tuple item is agent name.
        The second is the summ of agent scores.
        :return: array of sorted tuples, tournament result if in short
        """
        return [(item[0], sum([z.score for z in item[1].values()])) for item in self.get_winners_details()]

    def get_winner(self):
        """Gets the tournament winner.

        :param self: self
        :return: winner agent identifier
        """

        return "Tournament winner: " + str(self.get_winners_details()[0][0])


class Game:
    """Game value-object class (though without hash-code reimplementation)."""

    def __init__(self, one, loser, score):
        """Constructor.

        :param one: one agent
        :param loser: another agent
        :param score: score, positive, is the one agent is winner and negative if looser
        """
        self.one = one
        self.another = loser
        self.score = score

    def __repr__(self):
        """Gets representation for the debugging purposes.

        :return: string object representation
        """
        return self.to_string()

    def __str__(self):
        """Gets string object representation.

        :return: string object representation
        """
        return self.to_string()

    def to_string(self):
        """Gets string object representation.

        :return: string object representation
        """
        return "{one=" + str(self.one) \
               + "\nanother=" + str(self.another) \
               + "\nscore(positive, is one is winner, negative if looser)=" + str(self.score) + "}"

from kaggle_environments import evaluate


class TournamentRunner:
    """This is the runner for the Tournament class."""

    @staticmethod
    def get_score(agents, configuration, i, j):
        return evaluate(
            "rps",
            [agents[i], agents[j]],
            configuration)[0][0]

    @staticmethod
    def run(tournament, agents, configuration):
        """Runs the tournament for the given agents and configuration.

        Runs the tournament, prints the winner, winners list.

        :param tournament: Tournament, tournament implementation
        :param agents: list, agents participators
        :param configuration: map, kaggle environment configuration
        """

        for i in range(len(agents)):
            for j in range(i + 1, len(agents)):
                tournament.add_game(agents[i], agents[j], TournamentRunner.get_score(agents, configuration, i, j))

        print(tournament.get_winner(), "\n")

        winners = tournament.get_winners()
        [print(TournamentRunner.get_agent_result(i + 1, winners)) for i in range(len(winners))]

    @staticmethod
    def get_agent_result(position, winners):
        """Gets agent result string."""
        return str(position) + " place: " + str(winners[position - 1][0]) + ", score: " + str(winners[position - 1][1])

agents = [
       "scissors.py",
       "paper.py",
       "rock.py",
       "opponent_imitator.py",
       "uniform_random.py",
       "counter_attack_last_move.py",
       "anti_counter_attack_last_move.py",
       "anti_uniform_move.py",
       "mode.py",
       "anti_mode.py",
       "arithmetic_average_attack.py",
       "ablaze.py",
       "straight.py"
   ]
configuration = {"episodeSteps": 16,
                "tieRewardThreshold": 1}

TournamentRunner().run(Tournament(), agents, configuration)


Tournament winner: counter_attack_last_move.py 

1 place: counter_attack_last_move.py, score: 79.0
2 place: mode.py, score: 39.0
3 place: anti_uniform_move.py, score: 19.0
4 place: straight.py, score: 15.0
5 place: anti_counter_attack_last_move.py, score: 9.0
6 place: ablaze.py, score: 5.0
7 place: uniform_random.py, score: -8.0
8 place: anti_mode.py, score: -9.0
9 place: arithmetic_average_attack.py, score: -19.0
10 place: rock.py, score: -23.0
11 place: opponent_imitator.py, score: -24.0
12 place: paper.py, score: -39.0
13 place: scissors.py, score: -44.0


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [ ]:

from kaggle_environments import make, evaluate

evaluate(
    "rps", #environment to use - no need to change
    ["paper_agent.py", "rock_agent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[99.0, -99.0]]